In [2]:
from ipywidgets import *
import plotly.graph_objects as go
import numpy as np
import numpy as np
import scipy.fft as fft 
import scipy.signal as signal
import matplotlib.pyplot as plt
from math import *
##############################################################
delta_w = 50
T = 1
h = 0.0001
t_len = int(T/h)
ts = np.linspace(0,T,t_len)
################################################################


checkbox = widgets.Checkbox(
    value=False,
    description='set seed',
    disabled=False,
    indent=False,
    layout=Layout(width='100px',margin='0 0 0 30px')
)

slider = widgets.IntSlider(
    value=0,
    min=0,
    max=1,
    step=1,
)


seed_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='seed:',
    layout=Layout(width='300px')
)

fig1 = go.Figure( layout=go.Layout(
       font=dict(family="Courier New, monospace",size=18,color="#7f7f7f"),width=580, height=580,margin=dict(l=0, r=0, t=0, b=0) ,
    legend=dict(x=0, y=1)))

fig2 = go.Figure( layout=go.Layout(
       font=dict(family="Courier New, monospace",size=18,color="#7f7f7f"),width=580, height=580,margin=dict(l=0, r=0, t=0, b=0) ,
    legend=dict(x=0, y=1)))


In [33]:
c_x=0;
c_h=0;
def BFG1():
    
    global c_x
    c_x = np.exp((0 + 1j) * ( - delta_w * ts + 2.0 / T * delta_w * ts *ts))
    c_x_real = np.real(c_x)
    c_x_imag = np.imag(c_x)
    XX=[]
    for i in range(0,t_len):
        XX.append(i)
    
    return[[XX,c_x_real],[XX,c_x_imag]]
    
    
def BFG2():
    
    global c_h
    c_h = np.exp((0 - 1j) * ( - delta_w * (T - ts) + 2.0 / T * delta_w * (T - ts) * (T - ts)))
    c_h_real = np.real(c_h)
    c_h_imag = np.imag(c_h)
    
    XX = []
    for i in range(0,t_len):
        XX.append(i)
    return [[XX,c_h_real],[XX,c_h_imag]]

def BFG3():
    global c_h,c_x
    c_y = 0.5  * signal.convolve(c_h, c_x)
    c_y_real = np.real(c_y)
    c_y_imag = np.imag(c_y)
      
    XX = []
    for i in range(0,t_len):
        XX.append(i)
        
    return [[XX,c_y_real],[XX,c_y_imag]]


def BFG4():
    result = BFG2()
    Y = result[0][1]
    
    b,a = s.butter(3, 0.2, 'lowpass')
    
    XX = []
    for i in range(0,300):
        XX.append(i)
    
    Y = s.filtfilt(b, a, Y)
    
    return [[XX,Y], [XX,result[0][1]] ]


def BFG5():
    
    return [ [XX,X_new], [XX,X], [XX,filtered ], [XX,y_smooth ] ]




def css_widget(text, font_size=10):
    html = "<h1 style='font-size:"+str(font_size)+"px; margin-left: 50px; margin-top: 0;margin-bottom:0;width:350px'>" + str(text) + "</h1>"
    W = widgets.HTML(value=html)
    return W

In [34]:

step_btn = widgets.Button(description='refresh', button_style='primary')

def custom_layout(w,h):

    ll=Layout(width=str(w) + 'px', height=str(h) + 'px', border='solid')

    return ll 


def showText(text):
    print("\t", text)





def function(widget, figure, fun_for_data):
    
    figure.data = []       
    data = fun_for_data() 
 
    for values in data:
        figure.add_trace(go.Scatter(x=values[0], y=values[1]))
        
        

        


    figure.show()


def custom_display():
    
    text=""" На цифровой сигнал подается БГШ (белый гаусс. шум). Пропускаем через фильтр Баттерворта. Рассчитать отношения мощностей до и после фильтрации.
    В работе используются два фильтра: фильтр Баттерворта и встроенный в пакет scipy.signal lowess """
    
    
    ui = HBox([step_btn, checkbox, seed_slider], layout=Layout(margin='0 0 0 33px'))
    HH_ui1 = VBox([ui,interactive_plot1])
    HH_ui2 = VBox([ui,interactive_plot2])
    HH_ui3 = VBox([ui,interactive_plot3])
    HH_ui4 = VBox([ui,interactive_plot4])
    HH_ui5 = VBox([ui,interactive_plot5])
    
    
    tab = widgets.Tab(children = [HH_ui1, HH_ui2,HH_ui3,HH_ui4,HH_ui5 ])
    tab.set_title(0, '1')
    tab.set_title(1, '2')
    tab.set_title(2, '3')
    tab.set_title(3, '4')
    tab.set_title(4, '5')
    
    
    text_for_slider1= "ОТНОШЕНИЕ ЧИСТОГО К БАТТЕР"
    text_for_slider2= "ОТНОШЕНИЕ ЧИСТОГО К LOWESS"
    text_for_slider3= "ОТНОШЕНИЕ ЧИСТОГО К ШУМУ"

    
    

    
    HHH_2 = (Box([tab], layout = custom_layout(700,700)))
    
    
    
    Htext = VBox([css_widget("Задача 1",15),])
    
    
    HHH_3 = (Box([Htext], layout = custom_layout(500,700)))
    
    item_2 = HBox([HHH_2,HHH_3])

    display(VBox([item_2]))
    
    toggle_visibility(None)


def toggle_visibility(button):
    seed_slider.layout.visibility = 'hidden' if (checkbox.value == False) else  'visible'


def clicker(b):
    global SEED
    SEED = r.randrange(1, 100)
    slider.value = 1 if (slider.value == 0) else  0  
    custom_display()
       
step_btn.on_click(clicker)
checkbox.observe(toggle_visibility)

interactive_plot1 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG1) })
interactive_plot2 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG2) })
interactive_plot3 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG3) })
interactive_plot4 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG4) })
interactive_plot5 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG5) })

custom_display()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))